In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns

import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
dataset = pd.read_csv("../datasets/logisitc_regression/real_distribution_replicated_data_no_nulls (1).csv")

In [ ]:
df = dataset

In [ ]:
# Generate all pairs of rows for comparison
pairs = np.array(list(combinations(df.index, 2)))

# Calculate field similarity scores using TF-IDF and cosine similarity
def calculate_field_scores(df, pairs, field):
    vectorizer = TfidfVectorizer().fit(df[field])
    tfidf_matrix = vectorizer.transform(df[field])
    scores = [cosine_similarity(tfidf_matrix[pair[0]], tfidf_matrix[pair[1]])[0][0] for pair in pairs]
    return np.array(scores)

# Calculate similarity scores for selected fields
field_scores = {
    "name_score": calculate_field_scores(df, pairs, "ENGLISH_NAME"),
    "address_score": calculate_field_scores(df, pairs, "ADDRESS"),
    "email_score": calculate_field_scores(df, pairs, "EMAIL"),
    "website_score": calculate_field_scores(df, pairs, "WEBSITE")
}

# Function to get model input X and target variable y
def get_x_y(field_scores: dict[str, np.ndarray], pairs: np.ndarray, df: pd.DataFrame) -> tuple[pd.DataFrame, np.ndarray]:
    X = pd.DataFrame(field_scores)
    y = (df.loc[pairs[:, 0], "Cluster ID"].values == df.loc[pairs[:, 1], "Cluster ID"].values).astype(int)
    return X, y

# Generate X and y
X, y = get_x_y(field_scores, pairs, df)

# Display the results
print("Feature Matrix (X):")
print(X)
print("\nTarget Labels (y):")
print(y)